<a href="https://colab.research.google.com/github/danisoaresl/correlacao_regressao/blob/exercicio/correlacao_regressao_08_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Correlação e Regressão Simples/Multivariada

Desafio Final

Sugestões de bibliotecas para serem utilizadas:
Pandas, NumPy, Seaborn, Matplotlib, Statsmodels, Scipy

Considerando a base de dados kc_house_data.csv responda as questões de 1
a 6 abaixo:

1. Codifique e exiba a matriz de correlação entre as variáveis numéricas.
Além disso, qual é a correlação entre o preço price e o número de
quartos bedrooms? Existe alguma diferença na correlação quando
consideramos apenas casas com uma área total sqft_living superior a
2000 pés quadrados?

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

# Carregar os dados
df_ab = pd.read_csv('/content/drive/MyDrive/correlacao_regressao/kc_house_data.csv')

# 1. Descrição Estatística
print(df_ab.describe())

# 2. Verificar tipos de dados e colunas não numéricas
print(df_ab.dtypes)  # Mostra os tipos de dados de cada coluna

# Se necessário, converter colunas para o tipo correto ou remover colunas não numéricas
# Exemplo: se 'date' for uma coluna que causa problemas, você pode removê-la ou convertê-la
df_ab['date'] = pd.to_datetime(df_ab['date'], errors='coerce')  # Converte 'date' para datetime

# Verificar colunas após a conversão
print(df_ab.dtypes)

# 3. Matriz de Correlação
# Selecionar apenas colunas numéricas
correlation_matrix = df_ab.select_dtypes(include=['float64', 'int64']).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Matriz de Correlação')
plt.show()

# 4. Correlação entre price e bedrooms
correlation_price_bedrooms = df_ab['price'].corr(df_ab['bedrooms'])
print(f'Correlação entre preço e número de quartos: {correlation_price_bedrooms}')

# 5. Filtrar casas com sqft_living > 2000 e calcular a correlação novamente
filtered_houses = df_ab[df_ab['sqft_living'] > 2000]
correlation_price_bedrooms_filtered = filtered_houses['price'].corr(filtered_houses['bedrooms'])
print(f'Correlação entre preço e número de quartos para casas com sqft_living > 2000: {correlation_price_bedrooms_filtered}')

# 6. Gráficos de Dispersão
plt.figure(figsize=(12, 6))

# Todas as casas
plt.subplot(1, 2, 1)
sns.scatterplot(x='bedrooms', y='price', data=df_ab)
plt.title('Preço vs. Número de Quartos (Todas as Casas)')
plt.xlabel('Número de Quartos')
plt.ylabel('Preço')

# Casas com sqft_living > 2000
plt.subplot(1, 2, 2)
sns.scatterplot(x='bedrooms', y='price', data=filtered_houses)
plt.title('Preço vs. Número de Quartos (sqft_living > 2000)')
plt.xlabel('Número de Quartos')
plt.ylabel('Preço')

plt.tight_layout()
plt.show()

# 7. Histogramas de price e bedrooms
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df_ab['price'], bins=30, kde=True)
plt.title('Distribuição do Preço')
plt.xlabel('Preço')

plt.subplot(1, 2, 2)
sns.histplot(df_ab['bedrooms'], bins=10, kde=True)
plt.title('Distribuição do Número de Quartos')
plt.xlabel('Número de Quartos')

plt.tight_layout()
plt.show()

# 8. Boxplot para identificar outliers
plt.figure(figsize=(8, 5))
sns.boxplot(x='bedrooms', y='price', data=df_ab)
plt.title('Boxplot de Preço por Número de Quartos')
plt.xlabel('Número de Quartos')
plt.ylabel('Preço')
plt.show()


OBS.:Explicação do Resultado da Correlação entre Preço e Número de Quartos.

O coeficiente de correlação entre price e bedrooms para todas as casas é de aproximadamente 0.299, o que indica uma correlação positiva, mas não muito forte.
Isso significa que, em geral, à medida que o número de quartos aumenta, o preço tende a aumentar também, mas o número de quartos sozinho não é um fator determinante para o preço.
Já para as casas com sqft_living > 2000, a correlação é ainda menor (0.119), o que sugere que, para essas casas maiores, o número de quartos tem uma influência ainda mais reduzida no preço.

2. Existe alguma correlação entre o preço price e a área total da casa
sqft_living, considerando apenas casas com pelo menos dois
banheiros bathrooms?

OBS.:Explicação de como fazer: Podemos calcular a correlação entre price e sqft_living para casas que têm pelo menos dois banheiros (bathrooms >= 2). Para isso, vamos filtrar o DataFrame e, em seguida, calcular a correlação.

In [ ]:
# Filtrar casas com pelo menos dois banheiros
houses_with_two_bathrooms = df_ab[df_ab['bathrooms'] >= 2]

# Calcular a correlação entre preço e área total da casa
correlation_price_sqft = houses_with_two_bathrooms['price'].corr(houses_with_two_bathrooms['sqft_living'])
print(f'Correlação entre preço e área total da casa para casas com pelo menos dois banheiros: {correlation_price_sqft}')


OBS.: Explicação do resultado: Esse valor de 0.707 indica uma correlação positiva forte entre o preço (price) e a área total da casa (sqft_living) para casas com pelo menos dois banheiros. Em outras palavras, conforme a área total aumenta, o preço também tende a aumentar de forma significativa.

Esse resultado sugere que, para casas com dois ou mais banheiros, a área total (sqft_living) é um fator determinante no valor do imóvel, mais do que o número de quartos, que apresentou uma correlação mais fraca.

3. Como a quantidade de banheiros bathrooms influencia na correlação
entre a área total da casa sqft_living e o preço price?

OBS.:Explicação de como fazer: Para entender como a quantidade de banheiros influencia na correlação entre a área total da casa (sqft_living) e o preço (price), podemos analisar a correlação entre essas variáveis separadamente, para diferentes faixas de número de banheiros. Esse processo nos ajudará a identificar se a correlação se torna mais forte ou mais fraca conforme o número de banheiros aumenta.

Vamos dividir o dataset em grupos baseados no número de banheiros e calcular a correlação entre sqft_living e price em cada grupo:

In [ ]:
# Lista para armazenar os resultados de correlação para cada quantidade de banheiros
bathroom_corr = []

# Loop para calcular a correlação para cada valor único de 'bathrooms'
for bathrooms in sorted(df_ab['bathrooms'].unique()):
    # Filtra o dataframe para cada valor de 'bathrooms'
    filtered_df = df_ab[df_ab['bathrooms'] == bathrooms]
    # Calcula a correlação entre 'sqft_living' e 'price'
    corr = filtered_df['sqft_living'].corr(filtered_df['price'])
    # Adiciona o valor de correlação e o número de banheiros na lista
    bathroom_corr.append((bathrooms, corr))

# Exibe os resultados
for bathrooms, corr in bathroom_corr:
    print(f'Correlação entre sqft_living e price para {bathrooms} banheiros: {corr}')


OBS.: Explicação do resultado: Após obter esses valores, você pode interpretar da seguinte forma:

Se a correlação entre sqft_living e price aumenta com o número de banheiros, isso indica que, para casas maiores com mais banheiros, a área total influencia mais fortemente o preço.

Se a correlação diminui ou permanece a mesma, então a área total da casa tem um impacto mais consistente no preço, independentemente do número de banheiros.

Geralmente, ao aumentar o número de banheiros, é provável que a correlação entre sqft_living e price aumente, já que casas maiores com mais banheiros tendem a ser mais caras. Contudo, o resultado pode variar dependendo do dataset específico.

RESPOSTA: A quantidade de banheiros impacta diretamente a correlação entre a área total da casa (sqft_living) e o preço (price). Observa-se que:

Para poucas quantidades de banheiros (por exemplo, de 0.5 a 1.75 banheiros), a correlação é relativamente baixa, sugerindo uma conexão menos intensa entre a área total e o preço.

A partir de 2 banheiros, a correlação começa a aumentar, indicando uma associação mais forte entre a área total e o preço para casas com mais banheiros.

Em valores mais altos de banheiros (de 4.5 até 6 banheiros), a correlação se aproxima ou atinge 1.0, o que sugere que, para casas maiores e mais luxuosas, a área total é um fator altamente correlacionado com o preço.

Portanto, quanto mais banheiros uma casa possui, maior tende a ser a correlação entre a área total e o preço, o que sugere que casas maiores e de maior valor geralmente têm mais banheiros e que a área total é um importante fator de valorização dessas propriedades.

4. Qual é a relação entre a condição da casa condition e o preço price,
considerando apenas casas com uma área total sqft_living superior a
3000 pés quadrados?

OBS.:Para calcular a correlação:
1.Filtrar os Dados: Primeiro, selecione apenas as casas onde sqft_living > 3000.
2.Calcular a Correlação: Em seguida, calcule a correlação entre as colunas condition e price usando os dados filtrados.
3.Esse código fornecerá a correlação para que possamos entender a relação entre a condição da casa e o preço para imóveis de grande porte.

In [ ]:
# Filtrar as casas com área total superior a 3000 pés quadrados
casas_grandes = df_ab[df_ab['sqft_living'] > 3000]

# Calcular a correlação entre condição e preço para essas casas
correlacao_condition_price = casas_grandes['condition'].corr(casas_grandes['price'])
print("Correlação entre condition e price para casas com sqft_living > 3000:", correlacao_condition_price)


RESPOSTA: Correlação Positiva: O valor de 0.1053 sugere uma correlação positiva fraca entre a condição da casa e o preço. Isso significa que, embora haja uma tendência de que casas em melhores condições tendam a ter preços mais altos, a relação não é muito forte.

Significado Prático: Essa correlação fraca pode indicar que, para casas maiores, o preço pode ser influenciado mais por outros fatores, como a área total, localização ou características específicas da casa, do que pela condição da casa.

Resumo
A condição da casa tem uma influência relativamente pequena no preço para imóveis com área total acima de 3000 pés quadrados, sugerindo que os compradores podem priorizar outras características além da condição ao determinar o preço.



5. Existe alguma correlação entre a localização geográfica lat e long e o
preço price para casas com pelo menos três quartos bedrooms?

OBS.: Explicação de como fazer: 1.Filtrar as casas: Filtrar o DataFrame para incluir apenas casas com pelo menos três quartos.
2.Calcular a correlação: Calcular a correlação entre a latitude (lat), longitude (long) e o preço (price).

In [ ]:
# Filtrar as casas com pelo menos três quartos
casas_com_tres_quartos = df_ab[df_ab['bedrooms'] >= 3]

# Calcular a correlação entre latitude, longitude e preço
correlacao_lat_price = casas_com_tres_quartos['lat'].corr(casas_com_tres_quartos['price'])
correlacao_long_price = casas_com_tres_quartos['long'].corr(casas_com_tres_quartos['price'])

print("Correlação entre latitude e price para casas com pelo menos 3 quartos:", correlacao_lat_price)
print("Correlação entre longitude e price para casas com pelo menos 3 quartos:", correlacao_long_price)


OBS.: Interpretação dos Resultados:
Após executar o código, você obterá dois valores de correlação: um para a latitude e outro para a longitude em relação ao preço das casas.

Correlação Positiva: Um valor próximo de 1 sugere que, conforme a latitude ou longitude aumenta, o preço também tende a aumentar.

Correlação Negativa: Um valor próximo de -1 sugere que, conforme a latitude ou longitude aumenta, o preço tende a diminuir.

Correlação Próxima de Zero: Sugere que não há uma relação linear significativa entre a variável e o preço.

RESPOSTA: Correlação entre Latitude e Preço:

Valor: 0.3126 (aproximadamente)

Interpretação: Existe uma correlação positiva moderada entre a latitude e o preço das casas. Isso sugere que, à medida que a latitude aumenta, o preço tende a aumentar também, embora a relação não seja extremamente forte.

Correlação entre Longitude e Preço:

Valor: -0.0073 (aproximadamente)

Interpretação: A correlação entre a longitude e o preço é praticamente zero, o que indica que não há uma relação linear significativa entre essas duas variáveis. Assim, mudanças na longitude não parecem ter um impacto considerável sobre o preço das casas.

Resumo da Análise

A latitude tem uma relação positiva moderada com o preço, sugerindo que a localização geográfica (norte-sul) pode ser um fator que influencia os preços das casas.

A longitude, por outro lado, não demonstra uma relação significativa com o preço, indicando que a posição leste-oeste não parece afetar os preços de maneira consistente para casas com pelo menos três quartos.

6. Calcule a correlação entre uma variável categórica(waterfront) e uma
variável numérica(price) usando ANOVA.

OBS.:Explicação de como fazer:

Para calcular a correlação entre uma variável categórica, como waterfront, e uma variável numérica, como price, você pode usar a ANOVA (Análise de Variância).
A ANOVA é útil para verificar se há diferenças significativas nas médias da variável numérica entre os grupos definidos pela variável categórica.

Aqui está um passo a passo de como fazer isso:

Importar as bibliotecas necessárias.

Filtrar os dados se necessário.

Realizar a ANOVA usando o método apropriado (como o f_oneway da biblioteca scipy.stats).

Interpretar os resultados.

In [ ]:
import pandas as pd
from scipy import stats

# Supõe que df_ab é o seu DataFrame com os dados

# Converter a coluna waterfront
df_ab['waterfront_numeric'] = df_ab['waterfront'].map({'n': 0, 'y': 1})

# Separar os preços por grupos
preco_sem_waterfront = df_ab[df_ab['waterfront'] == 'n']['price']
preco_com_waterfront = df_ab[df_ab['waterfront'] == 'y']['price']

# Verificar contagem de casas em cada grupo
print("Contagem de casas sem waterfront:", preco_sem_waterfront.count())
print("Contagem de casas com waterfront:", preco_com_waterfront.count())

# Realizar ANOVA
f_stat, p_value = stats.f_oneway(preco_sem_waterfront, preco_com_waterfront)

print("Estatística F:", f_stat)
print("Valor-p:", p_value)

# Interpretação
if p_value < 0.05:
    print("Rejeitamos a hipótese nula: há diferença significativa entre os preços das casas com e sem waterfront.")
else:
    print("Não rejeitamos a hipótese nula: não há diferença significativa entre os preços das casas com e sem waterfront.")



OBS.: Interpretação dos Resultados

Contagem de Casas: A grande diferença nas contagens entre casas com e sem waterfront (9920 vs. 79) sugere que a maioria das casas não possui waterfront.


Estatística F: Um valor de F muito alto (964.57) indica que existe uma grande variação nos preços entre os dois grupos em comparação à variação dentro dos grupos. Isso sugere que a presença ou ausência de waterfront tem um impacto significativo nos preços.


Valor-p: O valor-p extremamente baixo (próximo de zero) indica que há uma probabilidade muito baixa de que as diferenças observadas sejam devidas ao acaso.


Conclusão: Como você rejeitou a hipótese nula, isso significa que há uma diferença significativa nos preços das casas com e sem waterfront. Em termos práticos, isso sugere que as casas com waterfront tendem a ter preços significativamente mais altos do que aquelas sem waterfront.

Calcular a Média e Mediana dos Preços.

Primeiro, você pode calcular a média e a mediana dos preços das casas com e sem waterfront.


In [ ]:
# Calcular a média e mediana dos preços
media_preco = df_ab.groupby('waterfront')['price'].mean()
mediana_preco = df_ab.groupby('waterfront')['price'].median()

print("Média dos preços por waterfront:")
print(media_preco)
print("\nMediana dos preços por waterfront:")
print(mediana_preco)


Criar Gráficos (Boxplots)

Em seguida, vamos criar um boxplot para visualizar a distribuição dos preços em relação à presença de waterfront.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Criar um boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='waterfront', y='price', data=df_ab)
plt.title('Distribuição dos Preços por Waterfront')
plt.xlabel('Waterfront (n = Sem, y = Com)')
plt.ylabel('Preço')
plt.show()


Analisar outras características das casas

Para entender melhor o que pode influenciar o preço, você pode analisar outras variáveis como bedrooms, sqft_living, condition, lat, long, etc.

Aqui está um exemplo para ver como o número de quartos (bedrooms) afeta o preço:

Calcular a Média de Preços por Número de Quartos

In [ ]:
# Calcular a média de preços por número de quartos
media_preco_bedrooms = df_ab.groupby('bedrooms')['price'].mean()
print("Média dos preços por número de quartos:")
print(media_preco_bedrooms)


Criar um Gráfico de Barras para Preços Médios por Número de Quartos

In [ ]:
# Criar um gráfico de barras
plt.figure(figsize=(10, 6))
media_preco_bedrooms.plot(kind='bar')
plt.title('Média dos Preços por Número de Quartos')
plt.xlabel('Número de Quartos')
plt.ylabel('Média do Preço')
plt.xticks(rotation=0)
plt.show()


Análise de Correlação

Você também pode calcular a correlação entre o preço e outras variáveis numéricas (por exemplo, sqft_living, condition, lat, long) para ver quais têm uma relação mais forte com o preço.

In [ ]:
# Verificar os tipos de dados no DataFrame
print(df_ab.dtypes)

# Converter a coluna 'waterfront' em numérica
df_ab['waterfront'] = df_ab['waterfront'].map({'n': 0, 'y': 1})

# Verifique se a conversão foi bem-sucedida
print(df_ab['waterfront'].unique())

# Calcular a correlação entre todas as variáveis
correlacoes = df_ab.corr()['price']
print("Correlação entre variáveis e preço:")
print(correlacoes)

# Selecionar apenas colunas numéricas
df_numerico = df_ab.select_dtypes(include=['number'])

# Calcular a correlação novamente
correlacoes_numericas = df_numerico.corr()['price']
print("Correlação entre variáveis numéricas e preço:")
print(correlacoes_numericas)



 Calcular a Média e Mediana dos Preços

Você pode calcular a média e mediana dos preços para casas com e sem waterfront. Isso ajudará a entender melhor a distribuição dos preços.

In [ ]:
# Calcular a média e mediana dos preços para casas com e sem waterfront
media_preco = df_ab.groupby('waterfront')['price'].mean()
mediana_preco = df_ab.groupby('waterfront')['price'].median()

print("Média dos preços:")
print(media_preco)
print("\nMediana dos preços:")
print(mediana_preco)


Criar Boxplots para Visualização

Boxplots são uma ótima maneira de visualizar a distribuição dos preços em relação à presença de waterfront. Você pode usar a biblioteca matplotlib ou seaborn para isso.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Criar um boxplot para visualizar a distribuição de preços
plt.figure(figsize=(10, 6))
sns.boxplot(x='waterfront', y='price', data=df_ab)
plt.title('Distribuição de Preços das Casas com e sem Waterfront')
plt.xlabel('Waterfront')
plt.ylabel('Price')
plt.xticks([0, 1], ['Sem Waterfront', 'Com Waterfront'])
plt.show()


Analisar Outras Características

Você também pode analisar como outras características das casas (como localização, número de quartos, etc.) influenciam o preço. Isso pode ser feito através de gráficos de dispersão ou análises de regressão.

Exemplo: Gráfico de Dispersão

In [ ]:
# Gráfico de dispersão entre sqft_living e price
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sqft_living', y='price', hue='waterfront', data=df_ab, alpha=0.7)
plt.title('Relação entre Área Total e Preço')
plt.xlabel('Área Total (sqft_living)')
plt.ylabel('Preço')
plt.legend(title='Waterfront', loc='upper left', labels=['Sem Waterfront', 'Com Waterfront'])
plt.show()


Considerando a base de dados car_price.csv responda as questões de 7 a 11
abaixo:

7. Codifique e exiba a Matriz de correlação para as variáveis numéricas e
dê exemplos de correlações positivas, negativas e neutras.

8. Codifique e exiba Gráficos de Dispersão para cada uma das variáveis
numéricas em relação à variável de interesse Price.

9. Crie um modelo de Regressão Linear Simples, exiba a Tabela de
Regressão e exiba o plot da Reta Estimada.

10. Codifique e exiba o gráfico dos resíduos do modelo de Regressão
Simples.

11. Crie um modelo de Regressão Multivariada, exiba a Tabela de Regressão
e exiba o gráfico dos resíduos do modelo.